<a href="https://colab.research.google.com/github/gromag/haystack-tutorials/blob/main/tutorials/Custom_1_%20OpenAIAnswerGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generative QA with "Retrieval-Augmented Generation"


While extractive QA highlights the span of text that answers a query,
generative QA can return a novel text answer that it has composed.
In this tutorial, you will learn how to set up a generative system using the
[RAG model](https://arxiv.org/abs/2005.11401) which conditions the
answer generator on a set of retrieved documents.

### Prepare environment

#### Colab: Enable the GPU runtime
Make sure you enable the GPU runtime to experience decent speed in this tutorial.
**Runtime -> Change Runtime type -> Hardware accelerator -> GPU**

<img src="https://raw.githubusercontent.com/deepset-ai/haystack/main/docs/img/colab_gpu_runtime.jpg">

You can double check whether the GPU runtime is enabled with the following command:

In [2]:
%%bash

nvidia-smi

Mon Feb  6 13:16:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0    28W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

To start, install the latest release of Haystack with `pip`:

In [3]:
%%bash

pip install --upgrade pip
pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,faiss]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 29.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-d10fmmhz/farm-haystack_7f4fea8c2b45460180a871e8625cb5c4
  Resolved https://github.com/deepset-ai/haystack.git to commit f4a30a552ae47ba11f3bb2d7350cf8703a352f30
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ━━━━━━━━━━━

DEPRECATION: git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,faiss] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-d10fmmhz/farm-haystack_7f4fea8c2b45460180a871e8625cb5c4


In [4]:
# Custom
!pip install python-dotenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import Open AI Keys

In order to use OpenAIAnswerGenerator we need to import the key which we will import from our mounted Google Drive

In [5]:
# Run this cell to connect to your Drive folder

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
from dotenv import dotenv_values

config = dotenv_values("gdrive/MyDrive/HayStack/openai.env")
OPEN_AI_KEY = config["OPEN_AI_KEY"]

## Logging

We configure how logging messages should be displayed and which log level should be used before importing Haystack.
Example log message:
INFO - haystack.utils.preprocessing -  Converting data/tutorial1/218_Olenna_Tyrell.txt
Default log level in basicConfig is WARNING so the explicit parameter is not necessary but can be changed easily:

In [7]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

Let's download a csv containing some sample text and preprocess the data.


In [8]:
import pandas as pd

from haystack.utils import fetch_archive_from_http


# Download sample
doc_dir = "data/tutorial7/"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/small_generator_dataset.csv.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

# Create dataframe with columns "title" and "text"
df = pd.read_csv(f"{doc_dir}/small_generator_dataset.csv", sep=",")
# Minimal cleaning
df.fillna(value="", inplace=True)

print(df.head())

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable  HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://docs.haystack.deepset.ai/docs/telemetry
INFO:haystack.utils.import_utils:Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/small_generator_dataset.csv.zip to 'data/tutorial7/'


               title  \
0  "Albert Einstein"   
1  "Albert Einstein"   
2  "Albert Einstein"   
3  "Albert Einstein"   
4     "Alfred Nobel"   

                                                                              text  
0  to Einstein in 1922. Footnotes Citations Albert Einstein Albert Einstein (; ...  
1  Albert Einstein Albert Einstein (; ; 14 March 1879 – 18 April 1955) was a Ge...  
2  observations were published in the international media, making Einstein worl...  
3  model for depictions of mad scientists and absent-minded professors; his exp...  
4  was adopted as the standard technology for mining in the "Age of Engineering...  


In [68]:
custom = {"title": "Giuseppe", "text":"Once upon a time, there was a little fox named Felix who lived in the forest. One day, while he was out exploring, he came across a beautiful tree with juicy red apples hanging from its branches. Felix was so excited to taste the sweet fruit, but no matter how hard he tried, he just couldn't reach them.Just then, a little child named Lily came wandering into the forest. Felix saw her and thought she might be able to help him reach the apples. So, he scampered over to her and asked for her assistance. Lily was more than happy to help. She climbed up the tree and plucked the apples for Felix. They both sat under the tree, munching on the delicious fruit and chatting away. From that day on, Felix and Lily became the best of friends. They went on many adventures together in the forest, and whenever Felix needed help, Lily was always there to lend a hand. And so, the little fox and the little child lived happily ever after, always grateful for their special friendship and the magic of the tree that brought them together."}
df.loc[len(df)] = custom

title                                                                           Giuseppe
text     Once upon a time, there was a little fox named Felix who lived in the forest...
Name: 75, dtype: object

In [69]:
df

,title,text
0,"""Albert Einstein""",to Einstein in 1922. Footnotes Citations Albert Einstein Albert Einstein (; ...
1,"""Albert Einstein""",Albert Einstein Albert Einstein (; ; 14 March 1879 – 18 April 1955) was a Ge...
2,"""Albert Einstein""","observations were published in the international media, making Einstein worl..."
3,"""Albert Einstein""",model for depictions of mad scientists and absent-minded professors; his exp...
4,"""Alfred Nobel""","was adopted as the standard technology for mining in the ""Age of Engineering..."
...,...,...
71,"""The Ashes""",1978–79; 1981; 1985; 1989; 1993 and 1997). Australians have made 264 centuri...
72,"""The Ashes""","Ground (MCG) (1876–77), and the Sydney Cricket Ground (SCG) (1881–82). A sin..."
73,"""The Ashes""",therefore will not host an Ashes Test until at least 2027. Trent Bridge is a...
74,"""The Ashes""",The Ashes The Ashes is a Test cricket series played between England and Aust...


We can cast our data into Haystack Document objects.
Alternatively, we can also just use dictionaries with "text" and "meta" fields

In [70]:
from haystack import Document


# Use data to initialize Document objects
titles = list(df["title"].values)
texts = list(df["text"].values)
documents = []
for title, text in zip(titles, texts):
    documents.append(Document(content=text, meta={"name": title or ""}))

Here we initialize the FAISSDocumentStore, DensePassageRetriever and RAGenerator.
FAISS is chosen here since it is optimized vector storage.

In [75]:
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import RAGenerator, DensePassageRetriever, OpenAIAnswerGenerator


# Initialize FAISS document store.
# Set `return_embedding` to `True`, so generator doesn't have to perform re-embedding
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat", return_embedding=True)

# Initialize DPR Retriever to encode documents, encode question and query documents
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
    use_gpu=True,
    embed_title=True,
)

# Initialize RAG Generator
generator = OpenAIAnswerGenerator(
    api_key=OPEN_AI_KEY, 
    model="text-davinci-003", 
    max_tokens=256, top_k=1, frequency_penalty=0, presence_penalty=0                                  
)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.model.language_model:Auto-detected model language: english
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.
INFO:haystack.modeling.model.language_model:Auto-detected model language: english


We write documents to the DocumentStore, first by deleting any remaining documents then calling `write_documents()`.
The `update_embeddings()` method uses the retriever to create an embedding for each document.


In [72]:
# Delete existing documents in documents store
document_store.delete_documents()

# Write documents to document store
document_store.write_documents(documents)

# Add documents embeddings to index
document_store.update_embeddings(retriever=retriever)

Writing Documents:   0%|          | 0/76 [00:00<?, ?it/s]

INFO:haystack.document_stores.faiss:Updating embeddings for 69 docs...


Updating Embedding:   0%|          | 0/69 [00:00<?, ? docs/s]

INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manu

Create embeddings:   0%|          | 0/80 [00:00<?, ? Docs/s]

Here are our questions:

In [17]:
QUESTIONS = [
    "who got the first nobel prize in physics",
    "when is the next deadpool movie being released",
    "which mode is used for short wave broadcast service",
    "who is the owner of reading football club",
    "when is the next scandal episode coming out",
    "when is the last time the philadelphia won the superbowl",
    "what is the most current adobe flash player version",
    "how many episodes are there in dragon ball z",
    "what is the first step in the evolution of the eye",
    "where is gall bladder situated in human body",
    "what is the main mineral in lithium batteries",
    "who is the president of usa right now",
    "where do the greasers live in the outsiders",
    "panda is a national animal of which country",
    "what is the name of manchester united stadium",
]

Now let's run our system!
The retriever will pick out a small subset of documents that it finds relevant.
These are used to condition the generator as it generates the answer.
What it should return then are novel text spans that form and answer to your question!

In [18]:
# Or alternatively use the Pipeline class
from haystack.pipelines import GenerativeQAPipeline
from haystack.utils import print_answers

pipe = GenerativeQAPipeline(generator=generator, retriever=retriever)
for question in QUESTIONS:
    res = pipe.run(query=question, params={"Generator": {"top_k": 1}, "Retriever": {"top_k": 5}})
    print_answers(res, details="minimum")

INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
  "error": {
    "message": "The server had an error while processing your request. Sorry about that!",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
, retry predict in 10.03 s


Query: who got the first nobel prize in physics
Answers:
[   {'answer': ' Albert Einstein'},
    {   'answer': ' Albert Einstein received the first Nobel Prize in Physics '
                  'in 1921.'}]


  "error": {
    "message": "The server had an error while processing your request. Sorry about that!",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
, retry predict in 10.19 seconds...
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_ke


Query: when is the next deadpool movie being released
Answers:
[   {'answer': ' The next Deadpool movie is not being released in 2020.'},
    {'answer': ' The next Deadpool movie is not being released in 2020.'}]


INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.


...


Query: who is the president of usa right now
Answers:
[   {'answer': ' The current President of the United States is Joe Biden.'},
    {'answer': ' The President of the United States is Joe Biden.'}]


INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.



Query: where do the greasers live in the outsiders
Answers:
[   {   'answer': ' The greasers in The Outsiders live in the city of Tulsa, '
                  'Oklahoma.'},
    {   'answer': ' The greasers in The Outsiders live in the lower-income '
                  'neighborhoods of Tulsa, Oklahoma.'}]


INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.



Query: panda is a national animal of which country
Answers:
[   {'answer': ' Panda is the national animal of China.'},
    {'answer': ' Panda is the national animal of China.'}]

Query: what is the name of manchester united stadium
Answers:
[{'answer': ' Old Trafford.'}, {'answer': ' Old Trafford.'}]


In [49]:
from haystack.nodes import TfidfRetriever

tfidf_retriever = TfidfRetriever(document_store=document_store)

INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manu

In [74]:
import pprint
question = "Where did the fox named Felix live?"
docs = tfidf_retriever.retrieve(query=question, top_k=3)
res = generator.predict(question, documents=docs, top_k=1)
pprint.pprint(res)

INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.


{'answers': [<Answer {'answer': ' Felix lived in the forest.', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_id': None, 'meta': {'doc_ids': ['ac80b44fb881e34689f1e7c4250bfc7d', '69c8b13a944a0878822e417f028aae49', '5e30f241d1c389b17b3e0d894589bb6d'], 'doc_scores': [None, None, None], 'content': ["Once upon a time, there was a little fox named Felix who lived in the forest. One day, while he was out exploring, he came across a beautiful tree with juicy red apples hanging from its branches. Felix was so excited to taste the sweet fruit, but no matter how hard he tried, he just couldn't reach them.Just then, a little child named Lily came wandering into the forest. Felix saw her and thought she might be able to help him reach the apples. So, he scampered over to her and asked for her assistance. Lily was more than happy to help. She climbed up the tree and plucked the apples for Felix. They both sat under the tree, 

In [80]:
question = "Why Lily and Felix became friends?"

pipe = GenerativeQAPipeline(generator=generator, retriever=retriever)
res = pipe.run(query=question, params={"Generator": {"top_k": 1}, "Retriever": {"top_k": 5}})
res

INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.


{'query': 'Why Lily and Felix became friends?',
 'answers': [<Answer {'answer': ' They became friends because Felix asked Lily for help to reach the apples in the tree and Lily was happy to help.', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_id': None, 'meta': {'doc_ids': ['ac80b44fb881e34689f1e7c4250bfc7d', 'd34ecd3aeacc8cbd33555e07f9a98c73', '4a347e0fc7c1b7d06fa7d7a2aa580555', '549e4d922dd841db8244599f1d9ac30c', '3c527839680f7f5ff3cb758454fa5079'], 'doc_scores': [0.6748286442320367, 0.6519696061470939, 0.6484747880080649, 0.6449249572628939, 0.6402925987478973], 'content': ["Once upon a time, there was a little fox named Felix who lived in the forest. One day, while he was out exploring, he came across a beautiful tree with juicy red apples hanging from its branches. Felix was so excited to taste the sweet fruit, but no matter how hard he tried, he just couldn't reach them.Just then, a little child named Lil

## About us

This [Haystack](https://github.com/deepset-ai/haystack/) notebook was made with love by [deepset](https://deepset.ai/) in Berlin, Germany

We bring NLP to the industry via open source!  
Our focus: Industry specific language models & large scale QA systems.  
  
Some of our other work: 
- [German BERT](https://deepset.ai/german-bert)
- [GermanQuAD and GermanDPR](https://deepset.ai/germanquad)
- [FARM](https://github.com/deepset-ai/FARM)

Get in touch:
[Twitter](https://twitter.com/deepset_ai) | [LinkedIn](https://www.linkedin.com/company/deepset-ai/) | [Discord](https://haystack.deepset.ai/community/join) | [GitHub Discussions](https://github.com/deepset-ai/haystack/discussions) | [Website](https://deepset.ai)

By the way: [we're hiring!](https://www.deepset.ai/jobs)